<h2 style='color:Green'>Traning the Model</h2>

In [1]:
import pandas as pd
import pickle
import numpy as np

#### Load the pickel file for level 3 - Training the Model

In [2]:
final_df = pickle.load(open('dataset_level2.pkl','rb'))

#### Split the dataset into Train and Test data

In [3]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [4]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
24070,England,West Indies,London,137,14,5,7.754717,33.0
57091,Australia,Sri Lanka,Pallekele,102,62,9,10.551724,45.0
36278,West Indies,England,Pallekele,108,50,9,9.257143,60.0
20755,New Zealand,England,Manchester,68,50,5,5.828571,34.0
51599,England,Pakistan,Sharjah,152,1,2,7.663866,52.0
...,...,...,...,...,...,...,...,...
40404,Australia,Pakistan,Dubai,53,55,5,4.892308,22.0
24984,New Zealand,West Indies,Hamilton,57,82,10,9.000000,43.0
6107,India,South Africa,Cape Town,68,69,9,8.000000,35.0
24197,Sri Lanka,New Zealand,Nottingham,140,11,6,7.706422,44.0


#### Import the relavent ML functions from Sklearn

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

#### Convert text data into numbers using one hot encoding(dummies) : batting_team, bowling_team, city

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

dfle = X
dfle.batting_team = le.fit_transform(dfle.batting_team)
dfle.bowling_team = le.fit_transform(dfle.bowling_team)
dfle.city = le.fit_transform(dfle.city)
dfle

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
46298,9,3,9,83,55,9,7.661538,36.0
22693,3,6,23,121,45,8,9.680000,41.0
5118,3,4,24,60,77,9,8.372093,40.0
31333,12,7,0,99,13,4,5.551402,31.0
23037,6,4,30,39,70,6,4.680000,30.0
...,...,...,...,...,...,...,...,...
44490,8,0,9,71,41,7,5.392405,20.0
16203,12,5,17,86,42,6,6.615385,31.0
15548,1,4,36,76,71,8,9.306122,36.0
39099,15,6,16,105,13,3,5.887850,40.0


In [7]:
X_ = dfle[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five']].values

In [8]:
X_

array([[ 9.        ,  3.        ,  9.        , ...,  9.        ,
         7.66153846, 36.        ],
       [ 3.        ,  6.        , 23.        , ...,  8.        ,
         9.68      , 41.        ],
       [ 3.        ,  4.        , 24.        , ...,  9.        ,
         8.37209302, 40.        ],
       ...,
       [ 1.        ,  4.        , 36.        , ...,  8.        ,
         9.30612245, 36.        ],
       [15.        ,  6.        , 16.        , ...,  3.        ,
         5.88785047, 40.        ],
       [14.        ,  6.        ,  5.        , ...,  9.        ,
         9.36363636, 55.        ]])

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('batting_team', OneHotEncoder(), [0])], remainder = 'passthrough')
ct1 = ColumnTransformer([('bowling_team', OneHotEncoder(), [0])], remainder = 'passthrough')
ct2 = ColumnTransformer([('city', OneHotEncoder(), [0])], remainder = 'passthrough')

In [10]:
X_ = ct.fit_transform(X_)
X_ = ct1.fit_transform(X_)
X_ = ct2.fit_transform(X_)
X_

array([[ 0.        ,  1.        ,  0.        , ...,  9.        ,
         7.66153846, 36.        ],
       [ 0.        ,  1.        ,  0.        , ...,  8.        ,
         9.68      , 41.        ],
       [ 0.        ,  1.        ,  0.        , ...,  9.        ,
         8.37209302, 40.        ],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  8.        ,
         9.30612245, 36.        ],
       [ 0.        ,  1.        ,  0.        , ...,  3.        ,
         5.88785047, 40.        ],
       [ 0.        ,  1.        ,  0.        , ...,  9.        ,
         9.36363636, 55.        ]])

#### from using GridsearchCV compare different models with different hyper parameters : XGBRegressor, linear_regression, lasso, DecisionTreeRegressor, RandomForestRegressor

In [11]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        
        'XGBRegressor': {
            'model': XGBRegressor(),
            'params': {
                'n_estimators' : [50,200,1000,2000],
                'max_depth': [6, 8, 12],
            }
        },
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'DecisionTreeRegressor': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'RandomForestRegressor': {
            'model': RandomForestRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'n_estimators': [20,100,200]
            }
        }
        

    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

trf = ColumnTransformer([
    (X,OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')
find_best_model_using_gridsearchcv(X_,y)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous 

,model,best_score,best_params
0,XGBRegressor,0.983864,"{'max_depth': 8, 'n_estimators': 2000}"
1,linear_regression,0.676975,{'normalize': True}
2,lasso,0.662392,"{'alpha': 1, 'selection': 'random'}"
3,DecisionTreeRegressor,0.943398,"{'criterion': 'friedman_mse', 'splitter': 'best'}"
4,RandomForestRegressor,0.977067,"{'criterion': 'friedman_mse', 'n_estimators': ..."


<h2 style='color:Red'>XGBRegressor is working perfectly as a model for the problem</h2>

#### Convert text data into numbers using one hot encoding(dummies) : batting_team, bowling_team, city

In [12]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

#### pipline for feature scaling and XGBRegressor

In [13]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=2000,learning_rate=0.2,max_depth=8,random_state=1))
])

### Train the Model with train data and find the score with test data

In [14]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9899774855898853
1.6797179834597455


<h2 style='color:Red'>Model gives a Accuracy of 0.99 and abs mean error of 1.68 which is pretty good</h2>

#### Export the final ML model as pickel file

In [15]:
pickle.dump(pipe,open('pipe.pkl','wb'))